In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
from ufc_fight_agent.tool import *
from ufc_fight_agent.ufc_data import *
from ufc_fight_agent.agent import Agent

In [3]:
@tool
def get_upcoming_matchups(
    max_events: Optional[int] = 1,
    title_fights_only: bool = False,
    weight_class: Optional[str] = None,
):
    """
    Fetches upcoming UFC fights with comprehensive fighter data and statistics.

    Parameters:
    - max_events: Limits the number of upcoming events to return (default: 1, which returns only the next upcoming event). Only increase this value when the user explicitly requests information about multiple events.
    - title_fights_only: When True, only returns championship fights
    - weight_class: Filter for a specific weight class (e.g., "Flyweight", "Bantamweight", "Featherweight", "Lightweight", "Welterweight", "Middleweight", "Light Heavyweight", "Heavyweight", "Women's Strawweight", "Women's Flyweight", "Women's Bantamweight")
    """
    upcoming_events = []

    events = extract_events(max_events=max_events)
    if not events:
        return upcoming_events

    for event in events:
        fights = extract_fights(event.event_url)
        filtered_fights = []

        for fight in fights:
            if title_fights_only and not fight.title_fight:
                continue
            if weight_class and fight.weight_class.lower() != weight_class.lower():
                continue
            fighter_1_stats, fighter_2_stats = extract_matchup(fight.fight_url)
            fight.fighter_1_stats = fighter_1_stats
            fight.fighter_2_stats = fighter_2_stats
            filtered_fights.append(fight)

        if filtered_fights:
            event.fights = filtered_fights
            upcoming_events.append(event)

    return upcoming_events

In [4]:
print(json.dumps(get_upcoming_matchups.to_claude_format(), indent=2))

{
  "name": "get_upcoming_matchups",
  "description": "Fetches upcoming UFC fights with comprehensive fighter data and statistics.",
  "input_schema": {
    "type": "object",
    "properties": {
      "max_events": {
        "type": "integer",
        "description": "Limits the number of upcoming events to return (default: 1, which returns only the next upcoming event). Only increase this value when the user explicitly requests information about multiple events.",
        "default": 1,
        "optional": true
      },
      "title_fights_only": {
        "type": "boolean",
        "description": "When True, only returns championship fights",
        "default": false,
        "optional": true
      },
      "weight_class": {
        "type": "string",
        "description": "Filter for a specific weight class (e.g., \"Flyweight\", \"Bantamweight\", \"Featherweight\", \"Lightweight\", \"Welterweight\", \"Middleweight\", \"Light Heavyweight\", \"Heavyweight\", \"Women's Strawweight\", \"W

In [5]:
agent = Agent(tools=[get_upcoming_matchups])

In [8]:
response_without_reasoning = agent.run(
    "Which upcoming UFC title fights are most likely to end in an upset based on the fighters' styles and recent performances?",
    thinking_mode=False,
)

========== USER QUERY ==========================================================
Which upcoming UFC title fights are most likely to end in an upset based on the fighters' styles and recent performances?

========== THINKING MODE =======================================================
Disabled

========== STEP 1 ==============================================================
Determining Tool Usage

========== TOOL SELECTED =======================================================
Using get_upcoming_matchups

========== STEP 2 ==============================================================
Executing Tool

========== EXECUTING TOOL ======================================================
{
  "name": "get_upcoming_matchups",
  "input": {
    "title_fights_only": true,
    "max_events": 5
  }
}

========== TOOL EXECUTION RESULT (PREVIEW) =====================================
[
  {
    "name": "UFC 314: Volkanovski vs. Lopes",
    "date": "April 12, 2025",
    "location": "Miami, Florida, USA",
  

In [9]:
response_without_reasoning = agent.run(
    "Which upcoming UFC title fights are most likely to end in an upset based on the fighters' styles and recent performances?",
    thinking_mode=True,
)

========== USER QUERY ==========================================================
Which upcoming UFC title fights are most likely to end in an upset based on the fighters' styles and recent performances?

========== THINKING MODE =======================================================
Enabled

========== STEP 1 ==============================================================
Determining Tool Usage

========== THINKING PROCESS ====================================================
The user is asking about upcoming UFC title fights that are most likely to end in an upset, based on the fighters' styles and recent performances. This is a perfect use case for the `get_upcoming_matchups` function with the `title_fights_only` parameter set to true.

Since the user is asking about title fights (plural), I should look beyond just the next event to find all upcoming title fights. I'll set `max_events` to 5 to get a good range of upcoming events, and `title_fights_only` to true to focus only on champi